<a href="https://colab.research.google.com/github/ElielBright/SlideGeneratore/blob/main/CompanionAISlides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers
!pip install python-pptx
!pip install nltk

from transformers import pipeline
from pptx import Presentation
from pptx.util import Pt, Inches
import re

# Initialize summarization pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

def preprocess_notes(notes):
    """
    Cleans and segments notes for processing.
    """
    cleaned_notes = re.sub(r'\s+', ' ', notes.strip())  # Remove unnecessary spaces and newlines
    segments = re.split(r'(?<=\n)', cleaned_notes)  # Split by newlines
    return [segment.strip() for segment in segments if segment.strip()]

def summarize_content(segments, max_len=100):
    """
    Summarizes each segment using an ML model.
    """
    summarized_segments = []
    for segment in segments:
        try:
            summary = summarizer(segment, max_length=max_len, min_length=50, do_sample=False)
            summarized_segments.append(summary[0]['summary_text'])
        except Exception as e:
            print(f"Error summarizing: {e}")
            summarized_segments.append(segment)  # Use original text if summarization fails
    return summarized_segments

def format_slide_content(content):
    """
    Formats content into structured bullet points for easy readability.
    """
    sentences = re.split(r'(?<=\.)\s', content)
    return "\n".join([f"- {sentence}" for sentence in sentences if sentence.strip()])

def generate_slide_deck(title, summarized_content, template_path=None):
    """
    Generates a PowerPoint slide deck with summarized content, optionally using a template.
    """
    # Load the presentation (use template if provided)
    try:
        presentation = Presentation(template_path) if template_path else Presentation()
    except Exception as e:
        print(f"Could not load template: {e}. Using default layout.")
        presentation = Presentation()

    # Title slide
    slide = presentation.slides.add_slide(presentation.slide_layouts[0])
    title_placeholder = slide.shapes.title
    subtitle_placeholder = slide.placeholders[1]
    title_placeholder.text = title
    subtitle_placeholder.text = "Automatically Generated Presentation"

    # Add content slides
    for content in summarized_content:
        slide = presentation.slides.add_slide(presentation.slide_layouts[5])  # Blank layout
        # Add custom title box
        title_box = slide.shapes.add_textbox(Inches(0.5), Inches(0.5), Inches(8), Inches(1))
        title_frame = title_box.text_frame
        title_frame.word_wrap = True
        title_frame.text = content.split(".")[0]  # First sentence as the slide title
        title_frame.paragraphs[0].font.size = Pt(24)
        title_frame.paragraphs[0].font.bold = True

        # Add content text box
        content_box = slide.shapes.add_textbox(Inches(0.5), Inches(1.5), Inches(8), Inches(5))
        content_frame = content_box.text_frame
        content_frame.word_wrap = True

        # Format content into bullet points
        slide_content = format_slide_content(content)
        for line in slide_content.split("\n"):
            p = content_frame.add_paragraph()
            p.text = line
            p.font.size = Pt(18)

    return presentation

def save_presentation(presentation, file_name="generated_presentation.pptx"):
    """
    Saves the generated presentation to a file.
    """
    presentation.save(file_name)
    print(f"Presentation saved as {file_name}")

def prepare_slides_from_notes(notes, title="Generated Slides", template_path=None):
    """
    Full pipeline: preprocess notes, summarize, and generate slides with an optional template.
    """
    print("Preprocessing notes...")
    segments = preprocess_notes(notes)

    print("Summarizing content...")
    summarized_content = summarize_content(segments)

    print("Generating slide deck...")
    presentation = generate_slide_deck(title, summarized_content, template_path)

    print("Saving presentation...")
    save_presentation(presentation)

# Example Usage
if __name__ == "__main__":
    # Example notes (can be replaced with actual input from the app)
    notes = """
    Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.
    Applications of AI include expert systems, natural language processing, speech recognition, and machine vision.
    AI can be categorized into weak AI (narrow AI) and strong AI (general AI).
    Weak AI is designed for a particular task, while strong AI has generalized human cognitive abilities.
    """
    # Path to the template file (set to None if you don't have one)
    template_path = None  # Replace with "custom_template.potx" if you have a template
    prepare_slides_from_notes(notes, title="Introduction to AI", template_path=template_path)


Device set to use cuda:0
Your max_length is set to 100, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)


Preprocessing notes...
Summarizing content...
Generating slide deck...
Saving presentation...
Presentation saved as generated_presentation.pptx


In [4]:
from google.colab import files
files.download("generated_presentation.pptx")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:

# Install required libraries
!pip install transformers python-pptx nltk joblib tqdm

# Import necessary libraries
from transformers import pipeline
from pptx import Presentation
from pptx.util import Pt, Inches
from pptx.dml.color import RGBColor
import re
import nltk
import logging
from joblib import Parallel, delayed
from google.colab import files  # For downloading files in Colab
from tqdm import tqdm  # For showing progress in parallel

# Initialize logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Download NLTK resources
# Download 'punkt_tab' instead of 'punkt'
nltk.download('punkt_tab')
nltk.download('stopwords')

# Initialize summarization pipeline
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")





# Preprocessing Function
def preprocess_notes(notes):
    cleaned_notes = re.sub(r'\s+', ' ', notes.strip())
    sentences = nltk.sent_tokenize(cleaned_notes)
    return [sentence.strip() for sentence in sentences if sentence.strip()]

# Summarization Helper
def summarize_segment(segment, max_len):
    try:
        segment_len = len(segment.split())
        dynamic_max_len = min(max(50, segment_len // 2), max_len)
        summary = summarizer(segment, max_length=dynamic_max_len, min_length=30, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        logging.error(f"Error summarizing segment: {e}")
        return segment

# Summarize Content with progress bar
def summarize_content(segments, max_len=100):
    # Using tqdm to show progress for parallel execution
    summarized_segments = Parallel(n_jobs=-1)(delayed(summarize_segment)(segment, max_len) for segment in tqdm(segments))
    return summarized_segments

# Format Slide Content
def format_slide_content(content):
    sentences = re.split(r'(?<=\.)\s', content)
    return "\n".join([f"- {sentence.strip()}" for sentence in sentences if sentence.strip()])

# Generate Slide Deck
def generate_slide_deck(title, summarized_content, template_path=None, custom_titles=None):
    try:
        presentation = Presentation(template_path) if template_path else Presentation()
    except Exception as e:
        logging.error(f"Could not load template: {e}. Using default layout.")
        presentation = Presentation()

    # Title slide
    slide = presentation.slides.add_slide(presentation.slide_layouts[0])
    title_placeholder = slide.shapes.title
    subtitle_placeholder = slide.placeholders[1]
    title_placeholder.text = title
    subtitle_placeholder.text = "Automatically Generated Presentation"

    # Add content slides
    for i, content in enumerate(summarized_content):
        slide = presentation.slides.add_slide(presentation.slide_layouts[1])
        title_placeholder = slide.shapes.title
        title_placeholder.text = custom_titles[i] if custom_titles and i < len(custom_titles) else f"Slide {i+1}"

        content_placeholder = slide.placeholders[1].text_frame
        content_placeholder.text = format_slide_content(content)

        # Adjust font size and color for content
        for paragraph in content_placeholder.paragraphs:
            paragraph.font.size = Pt(18)
            paragraph.font.color.rgb = RGBColor(0, 0, 0)

    return presentation

# Save Presentation
def save_presentation(presentation, file_name="generated_presentation.pptx"):
    presentation.save(file_name)
    logging.info(f"Presentation saved as {file_name}")

# Main Function
def prepare_slides_from_notes(notes, title="Generated Slides", template_path=None, custom_titles=None, max_len=100):
    logging.info("Preprocessing notes...")
    segments = preprocess_notes(notes)

    logging.info("Summarizing content...")
    summarized_content = summarize_content(segments, max_len=max_len)

    logging.info("Generating slide deck...")
    presentation = generate_slide_deck(title, summarized_content, template_path, custom_titles)

    logging.info("Saving presentation...")
    save_presentation(presentation)

    logging.info("Downloading presentation...")
    files.download("generated_presentation.pptx")

# Example Usage
if __name__ == "__main__":
    notes = """
    Artificial intelligence (AI) is the simulation of human intelligence processes by machines, especially computer systems.
    Applications of AI include expert systems, natural language processing, speech recognition, and machine vision.
    AI can be categorized into weak AI (narrow AI) and strong AI (general AI).
    Weak AI is designed for a particular task, while strong AI has generalized human cognitive abilities.
    """
    custom_titles = ["What is AI?", "Applications of AI", "Types of AI", "Weak vs Strong AI"]
    template_path = None  # Replace with your PowerPoint template if available
    prepare_slides_from_notes(notes, title="Introduction to AI", template_path=template_path, custom_titles=custom_titles)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Device set to use cuda:0
100%|██████████| 4/4 [00:32<00:00,  8.15s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>